## Load data

In [1]:
from langchain.docstore.document import Document
import os

folder_name = "data"

documents = []
for root, dirs, files in os.walk(folder_name):
    for file in files:
        try:
            with open(os.path.join(root, file), "r", encoding="utf-8") as o:
                code = o.readlines()
                d = Document(page_content="\n".join(code), metadata={"source": os.path.join(root, file)})
                documents.append(d)
        except UnicodeDecodeError:
            print("couldn't load ", file)
            pass

couldn't load  icon.png
couldn't load  logo.png
couldn't load  login_via_keycloak.png
couldn't load  login_with_openshift.png
couldn't load  Keycloak_Manageaccount.png
couldn't load  Keycloak_ChangePassword.png


In [2]:
len(documents)

783

## Store data in vector db

In [3]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

hfemb = HuggingFaceEmbeddings(model_name="krlvi/sentence-t5-base-nlpl-code-x-glue")
persist_directory = "db"
db = Chroma.from_documents(documents, hfemb, persist_directory=persist_directory)
db.persist()

## Load vector db

In [4]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

hfemb = HuggingFaceEmbeddings(model_name="krlvi/sentence-t5-base-nlpl-code-x-glue")

persist_directory = "db"
db = Chroma(persist_directory=persist_directory, embedding_function=hfemb)

## Vector Data retriever

In [5]:
#retriever = db.as_retriever(search_type="similarity_score_threshold", search_kwargs={"score_threshold": .2, "k": 5})

retriever = db.as_retriever()

## QA/Chat LLM model

In [6]:
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationSummaryMemory

from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.llms import GPT4All

# Callbacks support token-wise streaming
callbacks = [StreamingStdOutCallbackHandler()]

# Verbose is required to pass to the callback manager
llm = GPT4All(model="models/mistral-7b-openorca.Q4_0.gguf", callbacks=callbacks, verbose=False)

#memory = ConversationSummaryMemory(
#    llm=llm, memory_key="chat_history", return_messages=True
#)

## Setup chain

In [7]:
from langchain.chains import ConversationalRetrievalChain

qa_chain = ConversationalRetrievalChain.from_llm(llm=llm, retriever=retriever, return_source_documents=True, max_tokens_limit=2046)

## Query helper

In [8]:
def query(q):
    print(f"\nQuestion: {q}\n")
    result = qa_chain({"question":q, "chat_history":[]})
    #print(f"Answer: {result['answer']}")
    print(f"\n\nSources\n{[x.metadata['source'] for x in result['source_documents']]}")

## Chat

In [9]:
query("What do you know about argocd-operator?")


Question: What do you know about argocd-operator?



Token indices sequence length is longer than the specified maximum sequence length for this model (1802 > 1024). Running this sequence through the model will result in indexing errors


 The Argo CD Operator is a Kubernetes operator that manages the full lifecycle of Argo CD clusters, including installation, upgrading, backing up and restoring, as well as providing insights through Prometheus and Grafana. It aims to automate tasks required when operating an Argo CD cluster and provide easy configuration, seamless upgrades, backups, metrics exposure, and autoscaling capabilities.

Sources
['data/code/docs/index.md']


In [10]:
query("What is the source code langauge of argocd-operator?")


Question: What is the source code langauge of argocd-operator?

 The source code language for Argo CD Operator is Go.

Sources
[]


In [11]:
query("How to use this operator?")


Question: How to use this operator?

 To use the Argo CD Operator, follow these steps: 1) Install the operator using one of the methods described in the installation guide. This can be done through OpenShift console or manually if desired. 2) Once installed, create new `ArgoCD` resources to manage your applications and deployments

Sources
['data/code/docs/install/start.md', 'data/code/docs/install/openshift.md', 'data/code/build/bin/entrypoint']


In [12]:
query("How to install it on kubernetes?")


Question: How to install it on kubernetes?

 To manually install ArgoCD Operator, follow these steps: 1) Install a Kubernetes cluster using minikube or any other tool; 2) Download the source code from release section; 3) Modify the namespace in `config/default/kustomization.yaml` if needed; 4) Enable conversion webhook support (optional); 5) Deploy the operator using 'make deploy' command.

Sources
['data/code/docs/install/manual.md']


In [13]:
query("Can you read go.mod file of this project?")


Question: Can you read go.mod file of this project?

 Yes, you can read a Go.mod file in a project. It is a configuration file used by the Go programming language for managing dependencies and version control within a project.

Sources
[]


In [14]:
query("does argocd-operator source as go.mod file?")


Question: does argocd-operator source as go.mod file?

 Yes, it seems that argocd-operator uses a go.mod file to manage its dependencies.

Sources
['data/issues/462.txt']


In [15]:
query("can you read it?")


Question: can you read it?

 Yes, I can read the provided context.

Sources
['data/code/.readthedocs.yml', 'data/code/config/prometheus/kustomization.yaml', 'data/code/CONTRIBUTING.md', 'data/code/OWNERS']


In [16]:
query("Great. Tell me the go version in go.mod of argocd-opertator")


Question: Great. Tell me the go version in go.mod of argocd-opertator

 The current go version used by ArgoCD Operator is not mentioned explicitly, but it can be found in the go.mod file under the "go" directive.

Sources
['data/issues/462.txt']


In [17]:
query("Can you read it from 'go' directive and tell me?")


Question: Can you read it from 'go' directive and tell me?

 The 'go' directive is used in programming languages like Python or Ruby to indicate a function should be executed immediately without waiting for any return value. In this case, the text provided does not contain a 'go' directive, so I cannot answer your question based on that information alone.

Sources
[]


In [18]:
query("Can you read go version from 'go' directive in argocd-operator's go.mod file?")


Question: Can you read go version from 'go' directive in argocd-operator's go.mod file?

 Yes, the 'go' directive in a Go module specifies the minimum required Go version to build and run the code. In this case, it is possible to read the go version from the 'go' directive in argocd-operator's go.mod file.

Sources
['data/issues/462.txt', 'data/code/Dockerfile']


In [19]:
query("Then read and tell me?")


Question: Then read and tell me?

 I'm sorry, but as an AI model, I cannot provide PRs or make changes directly on GitHub. However, if you would like assistance with the content of your proposal, feel free to ask!

Sources
['data/code/docs/proposals/001-proposal-template.md', 'data/code/.readthedocs.yml', 'data/issues/679.txt']


In [20]:
query("Which sso does argocd-operator support?")


Question: Which sso does argocd-operator support?

 The Argo CD Operator currently supports Red Hat Single Sign-On (RH-SSO) version 7.5 by default, but it can be overridden in the operator group spec to use a different version or provider if needed.

Sources
['data/issues/274.txt', 'data/issues/153.txt', 'data/code/deploy/operator_group.yaml', 'data/issues/976.txt']


In [21]:
query("Does it support dex?")


Question: Does it support dex?

 Yes, the current operator supports Dex authentication with an external Dex server.

Sources
['data/issues/278.txt']


In [22]:
query("Give a example of dex with argocd-operator")


Question: Give a example of dex with argocd-operator

 

The DEX deployment should be configured with the correct command line arguments. For instance, in your ArgoCD configuration file (specified above), you have set `dex:` to use an image from quay.io/redhat-cop/dex and version v2.22.0-openshift. This is a valid example of DEX with argocd-operator.

Sources
['data/issues/306.txt']


In [ ]:
query("What is the latest version of ArgoCD CRD?")


Question: What is the latest version of ArgoCD CRD?

